In [ ]:
!pip -q install pyairtable

In [ ]:
from pyairtable import Api, Base, Table
import json
import pandas as pd
import numpy as np
import calendar
import gspread

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, drive
from oauth2client.client import GoogleCredentials

from google.auth import default
creds, _ = default()

# Authenticate GDrive -- this will ask for two authentication codes
auth.authenticate_user()
gauth = GoogleAuth()
gdrive = GoogleDrive(gauth)
gauth.credentials = GoogleCredentials.get_application_default()
drive.mount('/content/drive')

base_key = 'PUT_BASE_KEY_HERE'
api_key = 'PUT_API_KEY_HERE'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
table_month_year = {'January 2023': ['January', '2023'],
                    } #update this once next month airtable is available

def TableDF(api_key: str, base_id: str, table: str, month: str, year: str):
    # -- setup variables
    columns = {}
    table = Table(api_key, base_id, table)
    table_data = table.all()
    if len(table_data) > 0:
        # -- get columns
        columns = {}
        for col in table_data[0]["fields"].keys():
            columns[col] = []
    
        # -- populate dataframe
        df = pd.DataFrame(columns)
        for r, col in enumerate(table_data):
            entry = {}
            for col in table_data[r]["fields"].keys():
                entry[col] = table_data[r]["fields"][col]
            df = df.append(entry, ignore_index=True)
            df['Month'] = month
            df['Year'] = year
        return df
    else:
        return None

In [ ]:
appended_data = []

for key, value in table_month_year.items():
  df = TableDF(api_key, base_key, key, value[0], value[1] )
  appended_data.append(df)

appended_data = pd.concat(appended_data)

# Aggregate `appended_data` and then join with Net Worth Tracker

In [ ]:
condition = (appended_data['Category'] == 'Savings') & (appended_data['Status'] == 'Paid')
df = appended_data[condition]
df = df[['Net Worth Type', 'Transfer/Pay to/Pay via', 'Actual Amount', 'Goal', 'Salary Group', 'Month', 'Year']]

In [ ]:
rename_columns = {'Net Worth Type': 'Type', 'Transfer/Pay to/Pay via': 'Bank/Fund', 'Actual Amount': 'PHP'}
df.rename(columns=rename_columns, inplace=True)

In [ ]:
df.loc[df['Salary Group'] == '15th of the Month', 'Salary Group'] = '15'
df.loc[(df['Salary Group'] == '30th of the Month') & (df['Month'] != 'February'), 'Salary Group'] = '30'
df.loc[(df['Salary Group'] == '30th of the Month') & (df['Month'] == 'February'), 'Salary Group'] = '28'

In [ ]:
df['Date_As_Of'] = df['Month'] + ' ' + df['Salary Group'] + ', ' + df['Year']
df['Date_As_Of'] = pd.to_datetime(df['Date_As_Of'], format = '%B %d, %Y')
df['Date_As_Of'] = df['Date_As_Of'].astype(str)

In [ ]:
df

,Type,Bank/Fund,PHP,Goal,Salary Group,Month,Year,Date_As_Of


In [ ]:
df_aggregated_monthly = df.groupby(by=['Type', 'Bank/Fund', 'Month', 'Year']).agg(PHP = pd.NamedAgg(column='PHP', aggfunc='sum'),
                                                                                  Goal = pd.NamedAgg(column='Goal', aggfunc='max'),
                                                                                  Date_As_Of = pd.NamedAgg(column='Date_As_Of', aggfunc='max'),
).reset_index()

In [ ]:
rename_dateAsOf = {'Date_As_Of': 'Date As Of'}

df_aggregated_monthly.rename(columns=rename_dateAsOf, inplace=True)

#Open Net Worth Google Spreadsheet and Fuse It With Our Monthly Savings Tracker

In [ ]:
gc = gspread.authorize(creds)

wb = gc.open_by_url('PUT_NET_WORTH_GSHEET_LINK_HERE')
sheet = wb.worksheet('Savings Tracker')
data = sheet.get_all_values()

df = pd.DataFrame(data)
df.columns = df.iloc[0]
df.drop(index=0, inplace=True)

In [ ]:
df['PHP'] = df['PHP'].apply(lambda x: x.replace(',', ''))

In [ ]:
# Use this for tracking monthly savings

df_monthly = pd.concat([df_aggregated_monthly, df])
df_monthly['Date As Of'] = pd.to_datetime(df_monthly['Date As Of'], format = '%Y-%m-%d')
df_monthly['PHP'] = df_monthly['PHP'].astype(float)
df_monthly

,Type,Bank/Fund,Month,Year,PHP,Goal,Date As Of,USD
1,Emergency Fund,Seabank,NaN,NaN,142789.20,NaN,2022-12-25,
2,Stocks,COLFinancial,NaN,NaN,40369.09,NaN,2022-12-25,
3,Crypto,Crypto.com,NaN,NaN,13832.21,NaN,2022-12-25,250.47
4,Crypto,Ledger,NaN,NaN,6957.80,NaN,2022-12-25,125.99
5,Retirement Fund,PAGIBIG MP2,NaN,NaN,96664.84,NaN,2022-12-25,
6,Maintaining Balance,Security Bank,NaN,NaN,10787.92,NaN,2022-12-25,
7,Cash-in-Hand,Wallet,NaN,NaN,6300.00,NaN,2022-12-25,
8,Sabbatical Fund,TONIK,NaN,NaN,914.58,NaN,2022-12-25,


In [ ]:
# Use this for net worth dashboard

df_net_worth = df_monthly.groupby(by=['Type', 'Bank/Fund']).agg(PHP = pd.NamedAgg(column='PHP', aggfunc='sum'),
                                         Date_As_Of = pd.NamedAgg(column='Date As Of', aggfunc='max')
).reset_index()

df_net_worth['Date_As_Of'] = df_net_worth['Date_As_Of'].astype(str)
df_net_worth.rename(columns=rename_dateAsOf, inplace=True)

In [ ]:
# Update Net Worth Tracker Google Sheets. Remember that this is dependent with the `Savings Tracker` sheet.

sheet = wb.worksheet('Net Worth Tracker (DO NOT EDIT)')
sheet.update([df_net_worth.columns.values.tolist()] + df_net_worth.values.tolist())

{'spreadsheetId': '1iRaxeMevNjx_RN8rJd1WoVBKbKmsq_KL0G3xoAd42Bo',
 'updatedRange': "'Net Worth Tracker'!A1:D9",
 'updatedRows': 9,
 'updatedColumns': 4,
 'updatedCells': 36}

In [ ]:
sheet = wb.worksheet('Savings Goals 2023')
data = sheet.get_all_values()

df = pd.DataFrame(data)
df.columns = df.iloc[0]
df.drop(index=0, inplace=True)

In [ ]:
df_goal_tracker = pd.merge(df, df_net_worth, how='left', on=['Type', 'Bank/Fund'])
df_goal_tracker['Goal 2023'] = df_goal_tracker['Goal 2023'].apply(lambda x: x.replace(',', ''))
df_goal_tracker

,Type,Bank/Fund,Goal 2023,PHP,Date As Of
0,Emergency Fund,Seabank,184500.00,142789.20,2022-12-25
1,Stocks,COLFinancial,90369.09,40369.09,2022-12-25
2,Retirement Fund,PAGIBIG MP2,166664.84,96664.84,2022-12-25
3,Sabbatical Fund,TONIK,231361.00,914.58,2022-12-25


In [ ]:
sheet = wb.worksheet('Savings Goals 2023 Tracker (DO NOT EDIT)')
sheet.update([df_goal_tracker.columns.values.tolist()] + df_goal_tracker.values.tolist())

{'spreadsheetId': '1iRaxeMevNjx_RN8rJd1WoVBKbKmsq_KL0G3xoAd42Bo',
 'updatedRange': "'Savings Goals 2023 Tracker'!A1:E5",
 'updatedRows': 5,
 'updatedColumns': 5,
 'updatedCells': 25}